<a href="https://colab.research.google.com/github/AKonst2021/nlp2021/blob/main/Exam/Exam_Konstantynov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Практичне завдання

з дисципліни "Методи обробки природномовної інформації"

студента групи ТШІ-21

Константинова Артема Віталійовича

Варіант 5

Завдання:

Розробити Jupyter Notebook для вирішення завдання класифікації тексту
на основі набору даних:
[link]

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

Етап 1. Завантаження та попередній аналіз даних.

In [ ]:
#create the grouped dataset based on the data from each participant

In [19]:
!wget https://drive.google.com/drive/folders/1ygGIhh9mE06DK-RLJK6ai3uOVS0JLMSJ?usp=sharing

--2021-12-13 08:29:23--  https://drive.google.com/drive/folders/1ygGIhh9mE06DK-RLJK6ai3uOVS0JLMSJ?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.250.103.113, 142.250.103.102, 142.250.103.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.103.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘1ygGIhh9mE06DK-RLJK6ai3uOVS0JLMSJ?usp=sharing’

1ygGIhh9mE06DK-RLJK     [ <=>                ] 220.84K  1.29MB/s    in 0.2s    

2021-12-13 08:29:24 (1.29 MB/s) - ‘1ygGIhh9mE06DK-RLJK6ai3uOVS0JLMSJ?usp=sharing’ saved [226145]



In [27]:
#завантажуємо датасет
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
data_file = open("/content/drive/MyDrive/NLP/data.csv")
data_sentences = data_file.read().split('.')[:-1]

In [67]:
#Створюємо новий фрейм
new_df = []

#попередньо класифікаємо на real:1 та fake:0
for new in data_sentences:
  new_df.append({'Sentence':new, 'Label':'REAL'})

for new in data_sentences:
  new_df.append({'Sentence':new, 'Label':'FAKE'})

new_df = pd.DataFrame(data=new_df, columns=['Sentence', 'Label'])

new_df['Target'] = new_df['Label']
new_df.replace({'Target':{'REAL':1, 'FAKE':0}}, inplace=True)
new_df

,Sentence,Label,Target
0,",title,text,label\n8476,You Can Smell Hillary’...",REAL,1
1,"\nIn the final stretch of the election, Hilla...",REAL,1
2,\nThe word “unprecedented” has been thrown ar...,REAL,1
3,But it’s still unprecedented for the nominee ...,REAL,1
4,\nBut that’s exactly what Hillary and her peo...,REAL,1
...,...,...,...
506093,Those aren't Jeb Bush's people to begin with,FAKE,0
506094,He's trying to appeal to the other 60 percent...,FAKE,0
506095,"""""\n\nO'Connell agreed",FAKE,0
506096,He pointed out that even if this new approach...,FAKE,0


In [68]:
# векторизація тексту Векторизація тексту - це уявлення тексту як серії векторів, які можуть виражати семантику тексту
max_tokens = 10000

sentences = data_sentences
tokens_count = 0
for new in sentences:
  tokens_count+=len(new.split())
avg_tokens = round(tokens_count/len(sentences))

In [69]:
#токенізація сегментації тексту, що є послідовністю символів, таких як букви, пробіли, знаки пунктуації та цифри, на слова і фрази
# Вбудовування слів — це засвоєне уявлення для тексту, де слова, що мають однакове значення, мають подібне представлення.
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(new_df['Sentence'])

text_vectorizer(new_df['Sentence'])

<tf.Tensor: shape=(506098, 20), dtype=int64, numpy=
array([[   1,    1,   64, ...,   57,  244, 2576],
       [   7,    2,  763, ...,    0,    0,    0],
       [   2,  996,    1, ...,    0,    0,    0],
       ...,
       [   1, 1398,    0, ...,    0,    0,    0],
       [  15, 1382,   56, ...,    0,    0,    0],
       [   7,    6,  315, ...,    1,    2, 7924]])>

In [70]:
embedding = layers.Embedding(input_dim=max_tokens, # встановлюємо форму введення
                             output_dim=128, # встановлюємо розмір вбудовуваного вектору 
                             embeddings_initializer="uniform", # ініціалізовуємо вбудовування
                             input_length=avg_tokens) # встановлювана довжина

embedding(text_vectorizer(new_df['Sentence']))

<tf.Tensor: shape=(506098, 20, 128), dtype=float32, numpy=
array([[[-0.04027963,  0.03049413, -0.02688242, ..., -0.01163316,
         -0.04918681, -0.00197896],
        [-0.04027963,  0.03049413, -0.02688242, ..., -0.01163316,
         -0.04918681, -0.00197896],
        [ 0.01919878, -0.01921125,  0.01477207, ...,  0.00080711,
          0.03182292,  0.04635385],
        ...,
        [ 0.0397974 , -0.04884326, -0.03566861, ..., -0.0315243 ,
          0.04467866, -0.01481025],
        [ 0.01907578,  0.00651969, -0.00737723, ...,  0.02197366,
          0.03381652,  0.02662699],
        [-0.00426179, -0.00820217, -0.00339188, ..., -0.00394834,
         -0.00168412,  0.00298517]],

       [[ 0.02963964, -0.0104552 , -0.01239705, ...,  0.02225998,
          0.0161013 ,  0.01787629],
        [ 0.02376766,  0.03035039,  0.01386983, ...,  0.01934258,
         -0.04118548, -0.04786894],
        [ 0.00440338,  0.00040507, -0.03003628, ..., -0.00488923,
          0.00509734, -0.02167777],
        

2. Створення тренувального та тестувального наборів даних.

In [71]:
# Створюємо тестову та тренувальну вибірки
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(new_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Label,Target
18432,"But even if they did, it does not appear to h...",REAL,1
491789,They lied about the origins of the conflict,FAKE,0
317575,"” The other side says, “I know how bad you thi...",FAKE,0
24557,"The wars of regime change were, with few exce...",REAL,1
233241,\n\n-- A D,REAL,1
...,...,...,...
213789,"Yet the same strength in debating, self-aware...",REAL,1
21272,"\nThe couple in question, Faisal and Nazia Al...",REAL,1
487730,"\n\n""""Some say Abraham Lincoln might not have ...",FAKE,0
37569,Among them: Fisher v,REAL,1


In [72]:
#імпортуємо бібліотеки для класифікації
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

3. Попередня оцінка точності моделі за допомогою методу Naive Bayes.

In [73]:
#імпортуємо бібліотекуNaive Bayes Model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Створюємо токенізацію та моделюємо
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_split['Sentence'], train_split['Target'])
baseline_score = model_0.score(test_split['Sentence'], test_split['Target'])
print(f"Baseline model accuracy is: {baseline_score*100:.2f}%")

Baseline model accuracy is: 31.47%


4. Вирішення задачі класифікації за допомогою методу bidirectional GRU

In [74]:
#bidirectional GRU
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64))(x) # bidirectional працює в обидві сторони, тому має подвоєні параметри, ніж звичайний шар LSTM
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_gru = tf.keras.Model(inputs, outputs, name="model_Bi_gru")

In [75]:
# Компілюємо модель bidirectional GRU
model_bi_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_bi_gru.summary()

Model: "model_Bi_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 20)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 20, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,354,625
No

5. Графічне відображення тренування моделі.

In [76]:
model_bi_gru_history = model_bi_gru.fit(train_split['Sentence'],
                              train_split['Target'],
                              epochs=9,
                              validation_data=(test_split['Sentence'], test_split['Target']))

Epoch 1/9
12653/12653 [==============================] - 260s 20ms/step - loss: 0.6934 - accuracy: 0.4990 - val_loss: 0.6933 - val_accuracy: 0.4996
Epoch 2/9
12653/12653 [==============================] - 250s 20ms/step - loss: 0.6934 - accuracy: 0.5004 - val_loss: 0.6938 - val_accuracy: 0.4984
Epoch 3/9
12653/12653 [==============================] - 250s 20ms/step - loss: 0.6934 - accuracy: 0.5001 - val_loss: 0.6934 - val_accuracy: 0.4988
Epoch 4/9
12653/12653 [==============================] - 251s 20ms/step - loss: 0.6935 - accuracy: 0.4993 - val_loss: 0.6934 - val_accuracy: 0.4965
Epoch 5/9
12653/12653 [==============================] - 251s 20ms/step - loss: 0.6935 - accuracy: 0.4979 - val_loss: 0.6936 - val_accuracy: 0.4957
Epoch 6/9
12653/12653 [==============================] - 253s 20ms/step - loss: 0.6935 - accuracy: 0.5002 - val_loss: 0.6933 - val_accuracy: 0.5001
Epoch 7/9
12653/12653 [==============================] - 253s 20ms/step - loss: 0.6935 - accuracy: 0.4998 - val_

6. Порівняння результатів, отриманих у п.п. 3,4

In [79]:
model_bi_gru_pred_probs = model_bi_gru.predict(test_split['Sentence'])
model_bi_gru_preds = tf.squeeze(tf.round(model_bi_gru_pred_probs))
calculate_results(y_true=test_split['Target'], y_pred=model_bi_gru_preds)

{'accuracy': 50.29440821971942,
 'f1': 0.43465565815065754,
 'precision': 0.5036843194597989,
 'recall': 0.5029440821971942}

In [78]:
#compare model effectiveness
calculate_results(test_split['Target'], model_0.predict(test_split['Sentence']))

{'accuracy': 31.466113416320884,
 'f1': 0.31352159179622124,
 'precision': 0.3135020023554881,
 'recall': 0.31466113416320884}

**Висновок**

У даному завдані було виконано:
1. Завантаження та попередній аналіз даних.
2. Створення тренувального та тестувального наборів даних.
3. Попередню оцінку точності моделі за допомогою методу Naive Bayes.
4. Вирішення задачі класифікації за допомогою методу Bidirectional GRU.
5. Графічне відображення тренування моделі.
6. Порівняння результатів, отриманих у п.п. 3,4